In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from google.colab import drive

# 1. Connect to Google Drive and load models
drive.mount('/content/drive')

# Load models
model_lstm = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/backorder_lstm_model_dataset_3_2.h5')
model_gru = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/backorder_gru_model_dataset_3.h5')
model_cnn = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/backorder_cnn_model_dataset_3.h5')

# 2. Prepare test data
def create_sequences(data, seq_length=5):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length, :-1].values
        label = data.iloc[i+seq_length, -1]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 5
test_data = pd.read_csv('/content/drive/MyDrive/Thesis/Test_dataset_3.csv')
X_test, y_test = create_sequences(test_data, SEQ_LENGTH)

# 3. Get predictions from models
def get_predictions(model, X):
    y_pred = (model.predict(X) > 0.5).astype(int).flatten()
    y_proba = model.predict(X).flatten()
    return y_pred, np.column_stack([1-y_proba, y_proba])

lstm_pred, lstm_proba = get_predictions(model_lstm, X_test)
gru_pred, gru_proba = get_predictions(model_gru, X_test)
cnn_pred, cnn_proba = get_predictions(model_cnn, X_test)

# 4. Calculate model metrics
def calculate_metrics(y_true, y_pred, y_proba):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc': roc_auc_score(y_true, y_proba[:, 1]),
        'error_rate': 1 - accuracy_score(y_true, y_pred)
    }

models_results = {
    'LSTM': {
        'predictions': lstm_pred,
        'probabilities': lstm_proba,
        **calculate_metrics(y_test, lstm_pred, lstm_proba)
    },
    'GRU': {
        'predictions': gru_pred,
        'probabilities': gru_proba,
        **calculate_metrics(y_test, gru_pred, gru_proba)
    },
    'CNN': {
        'predictions': cnn_pred,
        'probabilities': cnn_proba,
        **calculate_metrics(y_test, cnn_pred, cnn_proba)
    }
}

# 5. Fixed Dempster-Shafer Fusion Class
class DempsterShaferFusion:
    def __init__(self, models_results):
        self.models_results = models_results
        self.models = list(models_results.keys())

    def calculate_model_weights(self):
        """Calculate weights based on model performance"""
        weights = {}
        for model in self.models:
            metrics = self.models_results[model]
            score = 0.4*metrics['auc'] + 0.3*metrics['f1'] + 0.2*metrics['accuracy'] + 0.1*metrics['precision']
            weights[model] = max(score, 0.1)  # Ensure minimum weight

        # Normalize weights
        total = sum(weights.values())
        return {model: weight/total for model, weight in weights.items()}

    def calculate_bpa(self):
        """Calculate Basic Probability Assignment (Fixed Version)"""
        bpa = {}
        weights = self.calculate_model_weights()
        n_samples = len(self.models_results[self.models[0]]['probabilities'])

        for model in self.models:
            probs = self.models_results[model]['probabilities']
            weight = weights[model]
            error_rate = self.models_results[model]['error_rate']

            # Initialize arrays
            believe_0 = np.zeros(n_samples)
            believe_1 = np.zeros(n_samples)
            uncertainty = np.zeros(n_samples)

            for i in range(n_samples):
                # Calculate belief masses
                believe_0[i] = (1 - probs[i, 1]) * (1 - error_rate) * weight
                believe_1[i] = probs[i, 1] * (1 - error_rate) * weight
                uncertainty[i] = error_rate * weight

                # Normalize
                total = believe_0[i] + believe_1[i] + uncertainty[i]
                if total > 0:
                    believe_0[i] /= total
                    believe_1[i] /= total
                    uncertainty[i] /= total

            bpa[model] = {
                'believe_0': believe_0,
                'believe_1': believe_1,
                'uncertainty': uncertainty
            }

        return bpa

    def combine_evidence(self, bpa):
        """Combine evidence using Dempster's rule"""
        n_samples = len(bpa[self.models[0]]['believe_0'])
        combined_0 = np.zeros(n_samples)
        combined_1 = np.zeros(n_samples)
        combined_uncertainty = np.zeros(n_samples)

        for i in range(n_samples):
            # Initialize with first model's BPA
            current_0 = bpa[self.models[0]]['believe_0'][i]
            current_1 = bpa[self.models[0]]['believe_1'][i]
            current_u = bpa[self.models[0]]['uncertainty'][i]

            # Combine with other models
            for model in self.models[1:]:
                m_0 = bpa[model]['believe_0'][i]
                m_1 = bpa[model]['believe_1'][i]
                m_u = bpa[model]['uncertainty'][i]

                # Calculate conflict
                K = current_0*m_1 + current_1*m_0

                if K < 1:  # Only combine if conflict is not total
                    # Combine beliefs
                    new_0 = (current_0*m_0 + current_0*m_u + current_u*m_0) / (1 - K)
                    new_1 = (current_1*m_1 + current_1*m_u + current_u*m_1) / (1 - K)
                    new_u = (current_u * m_u) / (1 - K)

                    # Normalize
                    total = new_0 + new_1 + new_u
                    if total > 0:
                        current_0, current_1, current_u = new_0/total, new_1/total, new_u/total

            combined_0[i] = current_0
            combined_1[i] = current_1
            combined_uncertainty[i] = current_u

        return combined_0, combined_1, combined_uncertainty

    def predict(self):
        """Make final predictions"""
        bpa = self.calculate_bpa()
        belief_0, belief_1, uncertainty = self.combine_evidence(bpa)

        predictions = (belief_1 > belief_0).astype(int)
        combined_probs = np.column_stack((belief_0, belief_1))

        return predictions, combined_probs, uncertainty

# 6. Apply Dempster-Shafer Fusion
ds_fusion = DempsterShaferFusion(models_results)
final_pred, final_proba, uncertainty = ds_fusion.predict()

# 7. Evaluate and save results
print("\nCombined Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, final_pred):.4f}")
print(f"Precision: {precision_score(y_test, final_pred):.4f}")
print(f"Recall: {recall_score(y_test, final_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, final_pred):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, final_proba[:, 1]):.4f}")

results_df = pd.DataFrame({
    'True_Label': y_test,
    'Final_Prediction': final_pred,
    'Probability_Class_0': final_proba[:, 0],
    'Probability_Class_1': final_proba[:, 1],
    'Uncertainty': uncertainty
})

# Add individual model results
for model in models_results:
    results_df[f'{model}_Pred'] = models_results[model]['predictions']
    results_df[f'{model}_Prob0'] = models_results[model]['probabilities'][:, 0]
    results_df[f'{model}_Prob1'] = models_results[model]['probabilities'][:, 1]

results_df.to_csv('/content/drive/MyDrive/Thesis/dempster_shafer_fixed_results.csv', index=False)

# Show model weights
weights = ds_fusion.calculate_model_weights()
print("\nModel Weights:")
for model, weight in weights.items():
    print(f"{model}: {weight:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Combined Model Performance:
Accuracy: 0.9769
Precision: 0.9629
Recall: 0.9440
F1-Score: 0.9533
AUC-ROC: 0.9961

Model Weights:
LSTM: 0.3321
GRU: 0.3334
CNN: 0.3345
